<a href="https://colab.research.google.com/github/JoaquinV13/UFV-Visualizacion/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install plotly_express

In [107]:
import plotly_express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from google.colab import drive
from statsmodels.tsa.seasonal import seasonal_decompose

In [4]:
drive.mount('/content/drive/')
data = pd.read_csv('/content/drive/MyDrive/Colab/datos_ejercicio_ventas.csv',sep=',')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
# Visualizamos los datos.
data.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


In [6]:
# Tipos de datos.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


In [7]:
# Número de 'Actuals' y 'AI_forecast'.
actuals = (sum(data['SCENARIO']=='actual'))
forecast = (sum(data['SCENARIO']=='AI_forecast'))
print('Número de Actuals:',actuals)
print('Número de Forecasts:',forecast)

Número de Actuals: 900
Número de Forecasts: 17766


In [8]:
# Número de regustros de cada marca por país.
tabla_pivot = data.pivot_table(index='COUNTRY', columns='SUBBRAND', aggfunc='size', fill_value=0)

# Agregar una columna con el total de registros por país.
tabla_pivot['Total'] = tabla_pivot.sum(axis=1)
print(tabla_pivot)

SUBBRAND       7up (L3)  7up Free (L3)  Lipton (L3)  Mountain Dew (L3)  \
COUNTRY                                                                  
Czech               236              7          236                236   
Denmark             216            236            0                236   
Great Britain       794            758          830                542   
Hungary             268            236          452                236   
Italy               236              0            0                  0   
Netherlands         236            236          452                216   
Norway              236            236          252                236   
Portugal            236            236          235                  0   
Spain               236            398          236                  0   

SUBBRAND       Pepsi Max (L3)  Pepsi Regular (L3)  Total  
COUNTRY                                                   
Czech                     236                 452   1403  
Denmark 

In [9]:
# Todos los datos del dataset tiene el mismo horizonte de previsión. Calculamos
# el de uno, sabemos el de todos.
data_1 = data[(data['FORECAST_YEAR']==2023) & (data['FORECAST']=='AI_P02F') & (data['COUNTRY']=='Portugal') & (data['SCENARIO']=='AI_forecast') & (data['SUBBRAND']=='Lipton (L3)')]
print(len(data_1))

18


In [10]:
# Comprobamos la cantidad de forecasts distintos que hay. Contando las diferentes
# fechas de inicio de un forecast.

data[(data['SCENARIO']=='AI_forecast')].loc[:, ['FORECAST_YEAR', 'FORECAST']].drop_duplicates().shape[0]

12

Vemos que la cantidad de forecast distintos es 12 pero, ¿esto es igual para todos los productos de cada país?

In [11]:
# Comprobamos que para cada producto de cada país se han hecho 12 estudios
# predicitivos teniendo en cuenta la fecha de inicio de la predicción.

Pais_producto = data[(data['SCENARIO']=='AI_forecast')].groupby(['COUNTRY','SUBBRAND'])

# Contar los valores únicos de la combinación de 'Year' y 'caca'
resultado = Pais_producto.apply(lambda x: x[['FORECAST_YEAR', 'FORECAST']].drop_duplicates().shape[0], include_groups=False).reset_index(name='Estudios')

print(resultado)

          COUNTRY            SUBBRAND  Estudios
0           Czech            7up (L3)        12
1           Czech         Lipton (L3)        12
2           Czech   Mountain Dew (L3)        12
3           Czech      Pepsi Max (L3)        12
4           Czech  Pepsi Regular (L3)        12
5         Denmark            7up (L3)        12
6         Denmark       7up Free (L3)        12
7         Denmark   Mountain Dew (L3)        12
8         Denmark      Pepsi Max (L3)        12
9         Denmark  Pepsi Regular (L3)        12
10  Great Britain            7up (L3)        12
11  Great Britain       7up Free (L3)        12
12  Great Britain         Lipton (L3)        12
13  Great Britain   Mountain Dew (L3)        12
14  Great Britain      Pepsi Max (L3)        12
15  Great Britain  Pepsi Regular (L3)        12
16        Hungary            7up (L3)        12
17        Hungary       7up Free (L3)        12
18        Hungary         Lipton (L3)        12
19        Hungary   Mountain Dew (L3)   

Efectivamente, para cada producto de cada país se han hecho 12 estudios de predicción distintos.

In [12]:
# Sumamos la cantidad de cada producto del país.
datos = data[(data['SCENARIO'] == 'actual')].groupby(['COUNTRY', 'SUBBRAND'])['AMOUNT'].sum().reset_index()
print(datos)

          COUNTRY            SUBBRAND        AMOUNT
0           Czech            7up (L3)  9.610990e+05
1           Czech       7up Free (L3)  3.242474e+05
2           Czech         Lipton (L3)  5.959059e+06
3           Czech   Mountain Dew (L3)  1.604077e+06
4           Czech      Pepsi Max (L3)  7.922810e+06
5           Czech  Pepsi Regular (L3)  1.858035e+07
6         Denmark       7up Free (L3)  6.356360e+05
7         Denmark   Mountain Dew (L3)  8.253904e+05
8         Denmark      Pepsi Max (L3)  4.936140e+07
9         Denmark  Pepsi Regular (L3)  5.774250e+06
10  Great Britain            7up (L3)  5.432014e+06
11  Great Britain       7up Free (L3)  2.451862e+07
12  Great Britain         Lipton (L3)  1.365866e+07
13  Great Britain   Mountain Dew (L3)  3.692249e+06
14  Great Britain      Pepsi Max (L3)  2.496323e+08
15  Great Britain  Pepsi Regular (L3)  3.784479e+07
16        Hungary            7up (L3)  1.418907e+05
17        Hungary       7up Free (L3)  1.244282e+06
18        Hu

In [13]:
# Normalizamos las cantidades de los productos diviendo entre el total de
# de cantidad vendida en el país.
country_totals = datos.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
country_totals = country_totals.rename(columns={'AMOUNT': 'COUNTRY_TOTAL'})

datos_N = pd.merge(datos, country_totals, on='COUNTRY', how='left')
datos_N['AMOUNT_N'] = datos_N['AMOUNT'] / datos_N['COUNTRY_TOTAL']

# Eliminamos columnas innecesarias.
datos_N.drop(columns=['AMOUNT', 'COUNTRY_TOTAL'], inplace=True)

print(datos_N)

          COUNTRY            SUBBRAND  AMOUNT_N
0           Czech            7up (L3)  0.027187
1           Czech       7up Free (L3)  0.009172
2           Czech         Lipton (L3)  0.168565
3           Czech   Mountain Dew (L3)  0.045375
4           Czech      Pepsi Max (L3)  0.224114
5           Czech  Pepsi Regular (L3)  0.525587
6         Denmark       7up Free (L3)  0.011231
7         Denmark   Mountain Dew (L3)  0.014584
8         Denmark      Pepsi Max (L3)  0.872161
9         Denmark  Pepsi Regular (L3)  0.102025
10  Great Britain            7up (L3)  0.016226
11  Great Britain       7up Free (L3)  0.073238
12  Great Britain         Lipton (L3)  0.040799
13  Great Britain   Mountain Dew (L3)  0.011029
14  Great Britain      Pepsi Max (L3)  0.745664
15  Great Britain  Pepsi Regular (L3)  0.113044
16        Hungary            7up (L3)  0.003416
17        Hungary       7up Free (L3)  0.029954
18        Hungary         Lipton (L3)  0.158036
19        Hungary   Mountain Dew (L3)  0

# Distribución de ventas realizadas en cada país.

In [96]:
fig = px.histogram(datos_N, x='COUNTRY', y='AMOUNT_N', color='SUBBRAND',
    title="Distribución de ventas realizadas por país",
    template="plotly_white",
    color_discrete_sequence=px.colors.qualitative.Prism, # Color pastel.
    height=700, # Modificamos la altura para que se vean bien las porporciones.
    width=1100)

fig.update_layout(xaxis_title="País",  yaxis_title="Proporción de las ventas",
    legend_title="Marca", title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
    bargap=0.4) # Ajustamos el espacio entre barras.

fig.show()

In [93]:
# Total de ventas por país.
fig = px.histogram(country_totals, x='COUNTRY', y='COUNTRY_TOTAL',
                   color='COUNTRY',
                   template="plotly_white",
                   color_discrete_sequence=px.colors.qualitative.Prism,
                   height=700,
                   width=1100)

fig.update_layout(title="Ventas/Cantidad total por país",
                xaxis_title="País", yaxis_title="Total de ventas/cantidad",
                showlegend=False,
                title_font=dict(size=24, color="black", family="Arial"),
                font=dict(size=16, color="black", family="Arial"))

fig.show()


# Distriución de ventas según mes y año.

In [16]:
# Número de combinaciones distintas de mes y año. Si lo conocemos sabremos
# cuántos puntos temporales necesitará nuestra gráfica.
combinaciones = data[['YEAR', 'MONTH']].drop_duplicates().values.tolist()
print(len(combinaciones))

29


In [17]:
data['copia_Marca'] = data['SUBBRAND']
Marca = data[data['SCENARIO'] == 'actual'].groupby('copia_Marca').apply(lambda x: x.sort_values(['YEAR', 'MONTH']),
       include_groups=False).reset_index(drop=True)
Marca.drop(columns=['COUNTRY','SCENARIO','FORECAST', 'FORECAST_YEAR'], inplace=True)
data.drop(columns=['copia_Marca'], inplace=True)
print(Marca)

               SUBBRAND  YEAR  MONTH         AMOUNT
0              7up (L3)  2023      1   64640.338704
1              7up (L3)  2023      1   10681.570661
2              7up (L3)  2023      1  146982.143738
3              7up (L3)  2023      1   78971.969596
4              7up (L3)  2023      1    1485.703014
..                  ...   ...    ...            ...
895  Pepsi Regular (L3)  2024      8  376188.333462
896  Pepsi Regular (L3)  2024      8  335232.153177
897  Pepsi Regular (L3)  2024      8  981788.232725
898  Pepsi Regular (L3)  2024      8  118063.302642
899  Pepsi Regular (L3)  2024      8  985290.247476

[900 rows x 4 columns]


In [18]:
# Sumamos el amount de aquellos productos que tengan la misma fecha de actual.
Marca = Marca.groupby(['YEAR', 'MONTH', 'SUBBRAND'])['AMOUNT'].sum().reset_index()
Marca['YEAR_MONTH'] = pd.to_datetime(Marca[['YEAR', 'MONTH']].assign(DAY=1))
print(Marca)

     YEAR  MONTH            SUBBRAND        AMOUNT YEAR_MONTH
0    2023      1            7up (L3)  7.411262e+05 2023-01-01
1    2023      1       7up Free (L3)  1.308115e+06 2023-01-01
2    2023      1         Lipton (L3)  3.399210e+06 2023-01-01
3    2023      1   Mountain Dew (L3)  2.883267e+05 2023-01-01
4    2023      1      Pepsi Max (L3)  1.485655e+07 2023-01-01
..    ...    ...                 ...           ...        ...
115  2024      8       7up Free (L3)  2.218120e+06 2024-08-01
116  2024      8         Lipton (L3)  4.825935e+06 2024-08-01
117  2024      8   Mountain Dew (L3)  4.248439e+05 2024-08-01
118  2024      8      Pepsi Max (L3)  2.273393e+07 2024-08-01
119  2024      8  Pepsi Regular (L3)  7.192413e+06 2024-08-01

[120 rows x 5 columns]


In [44]:
# Representamos las ventas de cada empresa en función del tiempo.
fig = px.line(Marca,
              x='YEAR_MONTH',
              y='AMOUNT',
              color='SUBBRAND',
              symbol="SUBBRAND",
              color_discrete_sequence=px.colors.qualitative.Safe,
              template="plotly_white",
              labels = {'AMOUNT':'Cantidad', 'YEAR_MONTH':'','SUBBRAND':'MARCA'},
              title='Ventas en función del tiempo')

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
)
fig.show()

# Distribución de ventas según la marca.

In [19]:
MarcaTotal = data[data['SCENARIO'] == 'actual'].groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
print(MarcaTotal)

             SUBBRAND        AMOUNT
0            7up (L3)  1.876851e+07
1       7up Free (L3)  3.427630e+07
2         Lipton (L3)  7.286515e+07
3   Mountain Dew (L3)  7.474391e+06
4      Pepsi Max (L3)  3.966987e+08
5  Pepsi Regular (L3)  1.398307e+08


In [74]:
fig = px.treemap(MarcaTotal,
                 path=['SUBBRAND'],
                 values='AMOUNT',
                 title='Cuota de mercado',
                 color_discrete_sequence=px.colors.qualitative.Safe,
                 color_continuous_scale='Blues')

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
    margin = dict(t=60, l=20, r=20, b=25))

fig.show()

In [92]:
# Se muestra un gráifco de bigotes para visualizar la distribución de ventas
# según la marca. Se limpian algunos outliers para visualizar mejor el gráfico.

datos11 = data[(data['SCENARIO'] == 'actual')]

# Elimino outliers.
datos11 = datos11[datos11['AMOUNT'] < 9000000]

fig = px.box(datos11, x='AMOUNT', y='SUBBRAND', color='SUBBRAND',
             color_discrete_sequence=px.colors.qualitative.Safe,
             template='plotly_white',
             labels={'AMOUNT': 'Cantidad', 'SUBBRAND': ''},
             notched=True, # used notched shape
             title='Distribución de las ventas por marca',
             hover_data=['YEAR','MONTH']
             )

fig.update_layout(showlegend=False,
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
)

fig.show()

#  Tendencia y estacionalidad

## Todas las ventas del país con menos ventas

In [105]:
# Sabemos por las gráficas anteriores que el país con menos ventas es España.
data_espana = data[(data['COUNTRY'] == 'Spain') & (data['SCENARIO'] == 'actual')]
data_espana = data_espana.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
data_espana['YEAR_MONTH'] = pd.to_datetime(data_espana[['YEAR', 'MONTH']].assign(DAY=1))

fig = px.area(data_espana, x="YEAR_MONTH", y="AMOUNT",
              labels={'YEAR_MONTH':'','AMOUNT':'Cantidad'},
              title='Total de ventas en España con respecto al tiempo',
              markers=True,
              color_discrete_sequence=px.colors.qualitative.Prism)

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"))

fig.update_yaxes(range=[0, 800000])

fig.show()

In [106]:
print(data_espana)

    YEAR  MONTH         AMOUNT YEAR_MONTH
0   2023      1  325272.938930 2023-01-01
1   2023      2  295588.379605 2023-02-01
2   2023      3  399710.273110 2023-03-01
3   2023      4  328035.257874 2023-04-01
4   2023      5  360751.156280 2023-05-01
5   2023      6  439198.586290 2023-06-01
6   2023      7  496545.236120 2023-07-01
7   2023      8  564336.120820 2023-08-01
8   2023      9  420041.980591 2023-09-01
9   2023     10  468425.841760 2023-10-01
10  2023     11  418006.199049 2023-11-01
11  2023     12  394147.408590 2023-12-01
12  2024      1  349696.533142 2024-01-01
13  2024      2  352899.035652 2024-02-01
14  2024      3  332109.256233 2024-03-01
15  2024      4  376678.368897 2024-04-01
16  2024      5  356759.011786 2024-05-01
17  2024      6  408967.746367 2024-06-01
18  2024      7  506234.362782 2024-07-01
19  2024      8  537862.283643 2024-08-01


In [111]:
result = seasonal_decompose(data_espana['AMOUNT'], model='additive', period=5)
px.line(result.trend, title='Tendencia')